<a href="https://colab.research.google.com/github/Sunkist2lee/EEE4178/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5(%EB%94%A5%EB%9F%AC%EB%8B%9D)%EA%B0%9C%EB%A1%A0_Project2_CNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

## Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
batch_size = 16
learning_rate = 0.001
num_epochs = 10

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
## 데이터 텐서 변환 및 정규화
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Load Data
train_data = torchvision.datasets.CIFAR10(root='./datasets',
                                        train=True,
                                        transform=transforms.ToTensor(),
                                        download=True)

test_data = torchvision.datasets.CIFAR10(root='./datasets',
                                       train=False,
                                       transform=transforms.ToTensor())

Files already downloaded and verified


In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                        shuffle=False, num_workers=2)


In [ ]:
## CIFAR10 DATASET 클래스 정의
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchsummary as summary ## layer 별 output shape 출력을 위해

class Net(nn.Module):
  def __init__(self, num_classes = 10):
      super(Net, self).__init__()

      #input = 3, 6 channel out, 5 * 5 kernal
      self.conv1 = nn.Sequential(
          nn.Conv2d(3, 6, 5, padding = 1),
          nn.BatchNorm2d(6),
          nn.ReLU(),
          nn.MaxPool2d((2,2),stride=2)
      )

      #input = 6. 16 channel out, 3 * 3 kernal
      self.conv2 = nn.Sequential(
          nn.Conv2d(6, 16, 3, padding = 1),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.MaxPool2d((2,2),stride=2)
      )

      #input = 16. 32 channel out, 3 * 3 kernal
      self.conv3 = nn.Sequential(
          nn.Conv2d(16, 32, 3, padding = 1), 
          nn.BatchNorm2d(32), 
          nn.ReLU(),
          nn.MaxPool2d((2,2),stride=2)
      )

      self.fc1 = nn.Linear(288, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      x = self.conv1(x)
      x = self.conv2(x)
      x = self.conv3(x)
      x = x.reshape(x.size(0),-1)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

model = Net(10).to(device) ## 인스턴스 

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
test_model = Net().to(device)

test_model.load_state_dict(torch.load("model.pth"))

test_model.eval()

with torch.no_grad():
  correct = 0
  
  for img, lab in testloader:
    img = img.to(device)
    lab = lab.to(device)
    out = test_model(img)
    _, pred = torch.max(out.data, 1)
    correct += (pred == lab).sum().item()

  print("Accuracy of the network on the {} test images: {}%".format(len(testloader)*batch_size, 100 * correct / (len(testloader) * batch_size)))


Accuracy of the network on the 10000 test images: 69.42%
